In [ ]:
import cPickle
import numpy as np
import graphlab as gl
import matplotlib.pyplot as plt
from IPython.display import display, Image
from scipy import ndimage

# Config the matlotlib backend as plotting inline in IPython
%matplotlib inline

In [ ]:
with open("mnist.pkl",'rb') as mnist_file:
    train_data,valid_data,test_data = cPickle.load(mnist_file)

In [ ]:
## In this block we are shuffling the data to avoid any classification bias
train_vec_in,train_label_out = train_data
permutation = np.random.permutation(train_vec_in.shape[0])
train_vec_in = train_vec_in[permutation,:]
train_reshape = np.reshape(train_vec_in,newshape=[50000,28,28])
train_label_out = train_label_out[permutation]

In [ ]:
valid_features,valid_labels = valid_data
test_features,test_labels = test_data

In [ ]:
def softmax_probability(coefficients,features):
    if np.shape(coefficients)[1] != np.shape(features)[0]:
        coefficients = np.transpose(coefficients)
    score = np.dot(coefficients,features)
    return np.transpose(np.exp(score-np.amax(score,axis=0))/np.sum(np.exp(score-np.amax(score,axis=0)),axis=0))

In [ ]:
## Coefficients = (num_classes,num_types_features)
#  features = (num_datapoints,num_types_features)
def softmax_regression(coefficients,features,output_label,classes,step_size,lamda,max_iter=1000,fit_intercept=True):
    print np.shape(coefficients)
    print np.shape(np.ones((len(classes),1)))
    if fit_intercept:
        features = np.concatenate((np.ones((np.shape(features)[0],1)),features),axis=1)
        coefficients = np.concatenate((np.ones((len(classes),1)),coefficients),axis=1)
    
    features_dim = np.shape(features)
    coeff_dim = np.shape(coefficients)
    probs = np.zeros(shape=(coeff_dim[1],features_dim[0]))
    cost = []
    
    for count in xrange(max_iter): 
        # Probability is of the shape = [number of datapoints, number of classes]
        probs = softmax_probability(coefficients,features.T)
        
        # For l2 regularization we will not take the sum of intercept coefficients.
        data_loss = -np.sum(np.log(probs[range(features_dim[0]),output_label])) + lamda*np.sum((coefficients*coefficients)[1:])
        cost.append(data_loss) #/features_dim[0])

        ''' Computing the gradient. 
        Shape of Features(X) = [Number of data points, number of features].
        Shape of Probabilities = [number of data points, number of classes].
        Here we will have to compute the outer product of X & Probabilities. Each column will then correspond to 
        the cost derivative w.r.t to corresponding class'''
        probs[range(features_dim[0]),output_label] -= 1
        derivative = np.dot(features.T,probs) + 2*lamda*coefficients.T
        
        ## Leaving out intercept from regularization
        coefficients[:,0] = coefficients[:,0] - step_size*(derivative[0,:])
        coefficients[:,1:] = coefficients[:,1:] - step_size*(derivative.T[:,1:] + 2*lamda*coefficients[:,1:])
    return coefficients,cost

In [ ]:
classes = set(train_label_out)
step_size = 1e-5
coefficients = np.zeros(shape=(len(classes),np.shape(train_vec_in)[1]))
model_coefficients,cost = softmax_regression(coefficients,train_vec_in,train_label_out,classes,step_size,lamda=0, \
                                             max_iter=100,fit_intercept=False)

In [ ]:
print "Accuracy on Training Set is {}%".format(compute_accuracy(model_coefficients,train_vec_in.T,train_label_out))
print "Accuracy on Test Set is {}%".format(compute_accuracy(model_coefficients,test_features.T,test_labels))
print "Accuracy on Validation Set is {}%".format(compute_accuracy(model_coefficients,valid_features.T,valid_labels))